<a href="https://colab.research.google.com/github/akhiljjj/Test1/blob/main/Final_Submission_Capstone_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip uninstall -y langchain langchain-classic langgraph langgraph-prebuilt
!pip install -q --force-reinstall "numpy<2.0"
!pip install -q \
  websockets==11.0.3 \
  torch==2.9.0 \
  torchvision==0.24.0+cu126 \
  torchaudio==2.9.0+cu126 \
  transformers>=4.38.0 \
  datasets \
  accelerate \
  sentence-transformers \
  faiss-gpu-cu12 \
  langchain-community \
  langchain-huggingface \
  langchain-core \
  gradio==3.50.2 \
  requests==2.32.4


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.27 requires langchain<1.0.0,>=0.3.26, which is not installed.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 wh

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from sentence_transformers import CrossEncoder
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [ ]:
print("📦 Loading dataset...")
dataset = load_dataset(
    "bitext/Bitext-customer-support-llm-chatbot-training-dataset",
    split="train"
)

print("📦 Building knowledge base...")
kb_docs = [
    Document(
        page_content=r["response"],
        metadata={"intent": r["intent"]}
    )
    for r in dataset.select(range(3000))
]

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_db = FAISS.from_documents(kb_docs, embeddings)

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


📦 Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess(examples):
    inputs = [f"support_agent: {x}" for x in examples["instruction"]]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        text_target=examples["response"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    labels_ids = [
        [(tok if tok != tokenizer.pad_token_id else -100) for tok in seq]
        for seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs


In [ ]:
tokenized_data = (
    dataset
    .select(range(3000))
    .map(preprocess, batched=True)
    .train_test_split(test_size=0.1)
)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_support_final",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    bf16=True,
    fp16=False,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

print("🚀 Fine-tuning started...")
trainer.train()


/tmp/ipython-input-718393061.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


🚀 Fine-tuning started...


Epoch,Training Loss,Validation Loss
1,No log,0.702614
2,No log,0.636190


Epoch,Training Loss,Validation Loss
1,No log,0.702614
2,No log,0.636190
3,0.817100,0.620269


TrainOutput(global_step=507, training_loss=0.8153769607844907, metrics={'train_runtime': 1811.5862, 'train_samples_per_second': 4.471, 'train_steps_per_second': 0.28, 'total_flos': 5546534554828800.0, 'train_loss': 0.8153769607844907, 'epoch': 3.0})

In [ ]:
def run_agent(message, history=None):
    try:
        # Normalize input
        query = message.get("text", "") if isinstance(message, dict) else str(message)
        if not query.strip():
            return "How can I help you today?"

        # Special business rule
        if "renew" in query.lower() and "subscription" in query.lower():
            return (
                "At the moment, I can help with newsletter subscriptions. "
                "For renewing paid subscriptions, please contact our associate."
            )

        # Escalation
        if any(k in query.lower() for k in ["sue", "legal", "lawyer", "court"]):
            return "⚠️ This issue requires human assistance. I’m escalating it now."

        # Intent map
        intent_map = {
            "cancel": ["cancel_order"],
            "track": ["track_order"],
            "where is my order": ["track_order"],
            "change order": ["change_order"],
            "modify order": ["change_order"],
            "replace": ["change_order"],
            "change address": ["change_shipping_address"],
            "refund": ["get_refund", "check_refund_policy"],
            "invoice": ["get_invoice"],
            "payment": ["payment_issue"],
            "create account": ["create_account"],
            "delete account": ["delete_account"],
            "password": ["recover_password"],
            "subscribe": ["newsletter_subscription"],
            "complaint": ["complaint"],
            "human": ["contact_human_agent"],
            "contact": ["contact_customer_service"],
        }

        detected_intent = None
        for k, v in intent_map.items():
            if k in query.lower():
                detected_intent = v
                break

        # Intent-aware retrieval (FAISS-safe)
        if detected_intent:
            filtered = [
                d for d in kb_docs if d.metadata["intent"] in detected_intent
            ]
            if filtered:
                temp_db = FAISS.from_documents(filtered, embeddings)
                docs = temp_db.similarity_search(query, k=5)
            else:
                docs = vector_db.similarity_search(query, k=5)
        else:
            docs = vector_db.similarity_search(query, k=5)

        # Rerank
        pairs = [[query, d.page_content] for d in docs]
        scores = reranker.predict(pairs)
        best_policy = docs[scores.argmax()].page_content

        # Prompt
        prompt = (
            "SYSTEM: You are a helpful customer support assistant.\n"
            f"POLICY: {best_policy}\n"
            f"CUSTOMER: {query}\n"
            "ASSISTANT:"
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=120,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
                repetition_penalty=2.0
            )

        return tokenizer.decode(output[0], skip_special_tokens=True)

    except Exception as e:
        return f"❌ Error: {e}"


In [ ]:
demo = gr.ChatInterface(
    fn=run_agent,
    title="Intelligent Customer Support Chatbot",
    description="Fine-tuned FLAN-T5 + RAG + Intent-Aware Retrieval",
    examples=[
        "Where is my order?",
        "Cancel my order",
        "Replace my order",
        "Renew my subscription",
        "I want to esclate this issue"
    ]
)

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://6a0c318b4f645f04d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6a0c318b4f645f04d3.gradio.live
